In [ ]:
from firebase import firebase
from datetime import datetime
import numpy as np

# get data
fb = firebase.FirebaseApplication(url)
data = fb.get('/trainingdata', None)
workouts = list(data.keys())[:]
# leave out workouts older than 90 days
for workout in workouts:
    if data[workout]['date'] < datetime.now().timestamp() - 90:
        workouts.remove(workout)
# filter data
data = np.array([[data[workout]['heart_rate'], data[workout]['power']] for workout in workouts])


# define GP models
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel, RBF

X = data[:, 1].reshape(-1, 1)
y = data[:, 0]

# Use the data to fit the HR(power) model
# Kernel:  1.52**2 * RBF(length_scale=121) + WhiteKernel(noise_level=0.13)
kernel = ConstantKernel(constant_value=1.52**2, constant_value_bounds='fixed') * RBF(length_scale=121, length_scale_bounds='fixed') + WhiteKernel(noise_level=0.13, noise_level_bounds='fixed')
gp1 = gaussian_process.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True)
gp1.fit(X, y)

# Initialize the HR(time) model
# Kernel:  0.78**2 * RBF(length_scale=8) + WhiteKernel(noise_level=0.001)
kernel2 = ConstantKernel(constant_value=0.78**2, constant_value_bounds='fixed') * RBF(length_scale=8, length_scale_bounds='fixed') + WhiteKernel(noise_level=0.001, noise_level_bounds='fixed')
gp2 = gaussian_process.GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=False)


In [ ]:
# simulating live data
# pwr as power, hr as heart rate, t as time
est_hr = 0
EST_HR = []
for i in range(len(t)):
    EST_HR.append(est_hr)
    # Get the predicted hr using HR(power) model
    x = pwr[i].reshape(-1, 1)
    mean_f, sigma = gp1.predict(x, return_std=True)
    # Fit HR(time) model using the predicted hr. Subtract mean from the predicted hr to converge towards the mean
    est_hrs_norm = np.array(EST_HR) - mean_f[0]
    gp2.fit(t[:i].reshape(-1, 1), est_hrs_norm.reshape(-1, 1))
    # Predict the HR curve using the HR(time) model (cover full workout length)
    est_hr_live = gp2.predict(t.reshape(-1, 1), return_std=False) + mean_f[0]
    # get standard deviation from HR(power) model
    _, sigma = gp1.predict(pwr.reshape(-1, 1), return_std=True)
    # update estimate
    est_hr = mean_f[0]

    # update plot
    update_live_plot(t, est_hr_live) # plot the HR(power) curve
    update_live_plot(t[:i], hr[:i], color='red') # plot the actual HR curve
    update_live_plot(t, est_hr_live, color='green', alpha=0.2, fill=True, sigma=sigma) # plot the standard deviation


